<a href="https://colab.research.google.com/github/DebbyLeung/rl_tutorial/blob/main/finetune_tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install tensorflow==2.14.0

Load dataset

In [5]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("glue", "cola")
dataset_train = dataset["train"]  # Just take the training split for now
dataset_sentence = dataset_train["sentence"]
dataset_label = dataset_train["label"]


Tokenized datasets

In [6]:
from transformers import AutoTokenizer
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenized_data = tokenizer(dataset_sentence, return_tensors="np", padding=True)
# Tokenizer returns a BatchEncoding, but we convert that to a dict for Keras
tokenized_data = dict(tokenized_data)
labels = np.array(dataset_label)  # Label is already an array of 0 and 1

In [7]:
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased")
model.compile(optimizer=Adam(3e-5))  # No loss argument!
# model.fit(tokenized_data, labels)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Branch 1

In [ ]:
model.fit(tokenized_data, labels)

In [54]:
model_path = "ft-bert-base-cased"
# model.save_pretrained(model_path)
model.save(model_path)

Branch 2

In [ ]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"])
dataset = dataset.map(tokenize_dataset)


In [ ]:
tf_dataset = model.prepare_tf_dataset(dataset["train"], batch_size=16, shuffle=True, tokenizer=tokenizer)

In [ ]:
model.compile(optimizer=Adam(3e-5))  # No loss argument!
model.fit(tf_dataset)